In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from gensim.models.doc2vec import TaggedDocument
from gensim.utils import simple_preprocess

NUM_PROGRAM = 8

cut_programs = np.load('cut_Programs.npy')
cut_questions = np.load('cut_Questions.npy')

voc_dict = np.load('voc_dict.npy')

#print(cut_programs[0][0])
sentences = []
tag = 0
for program in cut_programs:
    for episode in program:
        for sentence in episode:
            sentences.append(TaggedDocument(sentence, [tag]))
            tag = tag+1
            
#print(sentences[:5])    
print(len(sentences))         

2396906


In [3]:
print(cut_questions[0][0])

# 6 optional reponses
for i in range(1, 7):
    print(cut_questions[0][i])

[['媽給', '你', '送', '錢包', '來', '啦', ' ', '來', ' ', '你', '看', '一下', '是', '不', '是', '這個'], ['對', ' ', '就是', '這個', ' ', '你', '在', '哪裡', '找到', '它', '的'], []]
['你', '看', ' ', '這是', '我', '新', '買', '的', '錢包']
['我', '的', '錢包', '不見了', '啦']
['以後', '上', '網咖', '的', '錢包', '在', '我', '身上']
['什麼', '有', '錢包', '場']
['早上', '你', '爸爸', '在', '車上', '找到', '的', ' ', '一定', '是', '前天', '你', '放學', '的', '時候', '掉', '在', '車上', '了']
['我', '為什麼', '要給', '你們', '錢包']


In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim

model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(sentences)
%time model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)

In [18]:
model.infer_vector(['以後'])

array([-0.00655633,  0.0018341 ,  0.00909029, -0.00417535,  0.00287022,
       -0.00362734, -0.00428635, -0.00306576, -0.00326668, -0.00471139,
        0.00798222,  0.00580871, -0.00925604,  0.00779096,  0.00577432,
       -0.00578126,  0.00932818,  0.00676447, -0.00968587, -0.00824556,
        0.00057338,  0.00264536, -0.00826198,  0.00461383,  0.0072657 ,
        0.00798954,  0.0069545 , -0.00612866, -0.00465577, -0.00285655,
       -0.00909406, -0.00694388,  0.00142866, -0.00563768,  0.00063792,
       -0.00253888,  0.00361805, -0.00011609,  0.00840464, -0.00548477,
       -0.00716005,  0.00137779,  0.0016199 , -0.00828679, -0.00062136,
       -0.00482807,  0.00340087,  0.00994807,  0.00712933, -0.00318571],
      dtype=float32)

In [20]:
ranks = []
second_ranks = []
for doc_id in range(len(sentences)):
    inferred_vector = model.infer_vector(sentences[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

In [22]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(sentences[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(sentences[sims[index][0]].words)))

Document (4): «當然 豐富 啦»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3):

MOST (3, 0.12613166868686676): «這次 的 收穫 真 豐富»

SECOND-MOST (1, 0.04225558415055275): «昨天 晚上 的 流星雨»

MEDIAN (0, -0.015214674174785614): «還好 天氣 不錯»

LEAST (4, -0.22808095812797546): «當然 豐富 啦»

